In [2]:
import pandas as pd
import numpy as np
import pandas_profiling
import plotly.graph_objects as go
import plotly.express as px
from chord import Chord
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
import random
from sklearn import preprocessing
from matplotlib_venn import venn2
from matplotlib_venn import venn3

In [3]:
df = pd.read_csv('../../data_in/FINAL_TOP1000.csv',skipinitialspace=True, sep=";")


In [4]:
#change column data type to numeric
df["eval1"] = pd.to_numeric(df["eval1"])
df["eval2"] = pd.to_numeric(df["eval2"])
df["eval31"] = pd.to_numeric(df["eval31"])
df["eval32"] = pd.to_numeric(df["eval32"])
df["eval33"] = pd.to_numeric(df["eval33"])
df["LEV1"] = pd.to_numeric(df["LEV1"])
df["LEV2"] = pd.to_numeric(df["LEV2"])
df["LEV3"] = pd.to_numeric(df["LEV3"])
df

### Pre-Processing, noramlization of the dataframe
supset = df.filter(['LEV1','eval31'], axis=1)
supset['LEV1'] = supset.index
supset = supset.reset_index(drop=True)

scaler = preprocessing.MinMaxScaler()
names = supset.columns
d = scaler.fit_transform(supset)
scaled_df = pd.DataFrame(d, columns=names)
scaled_df

#group data set ds4 by ATC codes determined by algorithm 3 (similarity based on Levenshtein distance)
groupedATC = df.groupby('STEP31').count()
groupedATC = groupedATC.sort_values('eval31', ascending=False)

In [5]:
groupedATC

,MEDICATION,ATC_CODE,CAT_MATCH,size,PIPE,INGREDIENT,STEP1,STEP2,STEP32,STEP33,...,ATC_CODE_LVL1,eval1,eval2,eval31,eval32,eval33,correct,level,kommentar,Kommentar Apotheke
STEP31,,,,,,,,,,,,,,,,,,,,,
A02BC02,20,0,20,20,20,7,6,6,4,0,...,0,6,6,20,4,0,20,20,0,0
N02BB02,20,0,20,20,20,9,4,3,9,5,...,0,4,3,20,9,5,20,20,0,0
N05BA06,17,0,17,17,17,7,7,2,3,2,...,0,7,2,17,3,2,17,17,0,0
H02AB06,17,0,17,17,17,14,1,0,2,0,...,0,1,0,17,2,0,17,17,1,1
C03CA01,16,0,16,16,16,8,8,2,4,0,...,0,8,2,16,4,0,16,16,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
J01CG01,1,0,1,1,1,0,0,0,1,1,...,0,0,0,1,1,1,1,1,1,1
J01CR02,1,0,1,1,1,1,0,0,0,0,...,0,0,0,1,0,0,1,1,0,0
J01DH03,1,0,1,1,1,1,0,0,1,1,...,0,0,0,1,1,1,1,1,1,1


In [8]:
#create two new datasets for wrong and corret results of algorithm 3, preparation for t-test 
lev31FALSE = df.loc[df['eval31'] == 0.0]
lev31TRUE = df.loc[df['eval31'] == 1.0]

In [11]:
lev31TRUE["LEV1"].describe(include='all')

count    766.000000
mean      84.284595
std       14.856875
min       21.000000
25%       76.000000
50%       87.000000
75%       96.000000
max      100.000000
Name: LEV1, dtype: float64

In [12]:
lev31FALSE["LEV1"].describe(include='all')

count    234.000000
mean      67.183761
std       15.520594
min       29.000000
25%       55.000000
50%       63.000000
75%       75.000000
max      100.000000
Name: LEV1, dtype: float64

In [14]:
# Does the mean value of the Levensthein distance show a significant difference between TRUE and FALSE results - two sided t test 
# Levensthein Distance LEV1 belongs to results for STEP31 (algorithm 3, similarity match based on Levenshtein)
true = df[df['eval31']==1.0]
false = df[df['eval31']==0.0]
## two sided t-test to check if the two groups, true and falses differ significantly based on Lev1 distance value
ttest_ind(true['LEV1'], false['LEV1'], equal_var=False)

Ttest_indResult(statistic=14.897946966519024, pvalue=1.0360007176052793e-39)

In [15]:
#calculation the 95% confidence intervall for the ttest
mean_true = true["LEV1"].mean()
mean_false = false["LEV1"].mean()
diff_mean = mean_true - mean_false
print("mean true Lev1 für Step31: ", mean_true)
print("mean false Lev1 für Step31: ", mean_false)
print("diff mean true and fals: ", diff_mean)

mean true Lev1 für Step31:  84.2845953002611
mean false Lev1 für Step31:  67.18376068376068
diff mean true and fals:  17.10083461650042
